<a href="https://colab.research.google.com/github/Zywia/AD-projekt-2-/blob/additional_data/projekt_2_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert-for-tf2

     |████████████████████████████████| 40kB 3.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=b72e964b4d035600421b1ff46753598177a307532bc9ca06ee8e7f3981592902
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=3014ef12485bac6f7b36aa8d3300cab4ab0791689a590939b99bca3836803a16
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=66ea93647093ed26891511238cbe59279650ee682c0221da348816764f02c28d
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [2]:
!pip install tf-models-nightly

     |████████████████████████████████| 860kB 7.0MB/s 
     |████████████████████████████████| 358kB 39.2MB/s 
     |████████████████████████████████| 36.4MB 71kB/s 
     |████████████████████████████████| 102kB 8.6MB/s 
     |████████████████████████████████| 1.1MB 36.5MB/s 
     |████████████████████████████████| 174kB 40.2MB/s 
     |████████████████████████████████| 322.9MB 49kB/s 
     |████████████████████████████████| 296kB 39.6MB/s 
     |████████████████████████████████| 460kB 38.1MB/s 
     |████████████████████████████████| 3.0MB 40.0MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20069 sha256=7ba6a13aec1ef6e04ab8e830c31ba797000428522cb35c55664013cae395d173
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=ce0d887198e9970affd9a53941f7a329cb4c09bbcbdc1e0d94cb2d2df931ef0e
  Stored in directory: /

In [3]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import pickle
import pandas as pd
# from bert.tokenization
from bert import bert_tokenization
import official.nlp.bert.tokenization
# from bert.tokenization import FullSentencePieceTokenizer


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/1",
                            trainable=True)

In [6]:
train_data, test_data = tfds.load(name="civil_comments", split=["train", "test"], data_dir="/content/drive/My Drive/AD/civil_comments", batch_size=-1, as_supervised=True)



In [39]:
train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

In [40]:
more_to_train_negative = pd.read_csv("more.csv")
more_to_test_negative = pd.read_csv("test_n.csv")

In [41]:
len( more_to_test_negative.comment_text.to_list() )

6229

In [42]:
negative_comments_positions = np.argwhere(train_labels == 1)[:,0]
negative_comments = train_examples[negative_comments_positions]
negative_comments = np.hstack((negative_comments,  more_to_train_negative.comment_text.to_numpy() ) )


negative_comments_positions_test = np.argwhere(test_labels == 1)[:,0]

negative_comments_test = test_examples[negative_comments_positions_test]
negative_comments_test = np.hstack((negative_comments_test,  more_to_test_negative.comment_text.to_numpy() ) )





In [43]:

not_all_positive_comments_positions = np.argwhere(train_labels == 0)[:negative_comments.shape[0], 0]
not_all_positive_comments_positions_test = np.argwhere(test_labels == 0)[:negative_comments_test.shape[0], 0]

not_all_positive_comments = train_examples[not_all_positive_comments_positions]
not_all_positive_comments_test = test_examples[not_all_positive_comments_positions_test]


print( not_all_positive_comments_positions_test.shape)
print( not_all_positive_comments_positions.shape)

(6451,)
(20577,)


In [44]:
labels_T = np.hstack( (np.ones(negative_comments.shape[0]), np.zeros(negative_comments.shape[0]) ) )
labels_t = np.hstack( (np.ones(negative_comments_test.shape[0]), np.zeros(negative_comments_test.shape[0]) ) )


all_to_train = np.hstack((negative_comments, not_all_positive_comments ))
all_to_test = np.hstack(( negative_comments_test, not_all_positive_comments_test))



perm_T = np.random.permutation(all_to_train.shape[0])
perm_t = np.random.permutation(all_to_test.shape[0])

train_examples = all_to_train[perm_T]
train_labels = labels_T[perm_T]

test_examples = all_to_test[perm_t]
test_labels = labels_t[perm_t]

In [45]:
type (test_examples )

numpy.ndarray

In [46]:
more_to_training = 8000


train_examples = np.hstack((train_examples, test_examples[:more_to_training] ))
train_labels = np.hstack((train_labels, test_labels[:more_to_training] ))

test_examples = test_examples[more_to_training:]
test_labels = test_labels[more_to_training:]


In [48]:
test_examples.shape

(4902,)

In [51]:
idx = 41

print (test_labels[idx] )
print (test_examples[idx])

0.0
b"That's an odd comment, CanadaRob3, especially given the facts.  However, no odder than 'The media lies - to YOU' liking it and 'disagreeing' with reports of what the judge actually did.  I'll wait for  your responses, shall I?"


In [52]:
?official.nlp.bert.tokenization.FullSentencePieceTokenizer

In [53]:
sp_model_file = bert_layer.resolved_object.sp_model_file.asset_path.numpy()
tokenizer = official.nlp.bert.tokenization.FullSentencePieceTokenizer(sp_model_file)


def tokenize_text(txt):
    tokenized_txt = tokenizer.tokenize(txt)
    tokenized_txt = ["[CLS]"] + tokenized_txt + ["[SEP]"]
    tokenized_txt = tokenizer.convert_tokens_to_ids(tokenized_txt)
    return tokenized_txt

In [54]:

train_examples = list(map(tokenize_text, train_examples))
test_examples = list(map(tokenize_text, test_examples))

In [71]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_examples, maxlen=125, padding='post')
max_seq_length = x_train.shape[1]
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_examples, maxlen=max_seq_length, padding='post', truncating='post')

train_mask = (x_train != 0).astype(int)
test_mask = (x_test != 0).astype(int)

train_segments = np.zeros(x_train.shape)
test_segments = np.zeros(x_test.shape)

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_inputs = [input_word_ids, input_mask, segment_ids]
pooled_output, sequence_output = bert_layer(bert_inputs)

final_layer = tf.keras.layers.Dense(1, activation="sigmoid")
final_output = final_layer(pooled_output)
# tf.keras.layers.


In [72]:
model = tf.keras.models.Model(inputs=bert_inputs, outputs=final_output)

In [73]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.optimizers.Adam(learning_rate=1e-5), 
              # optimizer=tf.optimizers.Adam(), 

              metrics=["accuracy"])

In [74]:
model.fit([x_train, train_mask, train_segments], train_labels, epochs=4, batch_size=32,
          validation_data=([x_test, test_mask, test_segments], test_labels))

Epoch 1/4
1537/1537 [==============================] - 2197s 1s/step - loss: 0.1244 - accuracy: 0.9554 - val_loss: 0.1664 - val_accuracy: 0.9396
Epoch 2/4
1537/1537 [==============================] - 2209s 1s/step - loss: 0.0858 - accuracy: 0.9698 - val_loss: 0.1785 - val_accuracy: 0.9317
Epoch 3/4
1537/1537 [==============================] - 2210s 1s/step - loss: 0.0604 - accuracy: 0.9795 - val_loss: 0.1628 - val_accuracy: 0.9370
Epoch 4/4
1537/1537 [==============================] - 2183s 1s/step - loss: 0.0451 - accuracy: 0.9851 - val_loss: 0.1792 - val_accuracy: 0.9368


In [76]:
import pandas as pd

text = pd.read_csv("notI.csv", names=["sentance"], index_col=False)
text = text.to_numpy()

print (text[:,0].tolist())
text = text[:,0].tolist()
text = list(map(tokenize_text, text))
print(text[0])
text = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=100, padding='post')
mask = (text != 0).astype(int)
segments = np.zeros(text.shape)


['temporrytext that is absoulutlely not important', 'anotehre aboulutally not important text', 'you are ugly so ugly', 'hate speech']
[2, 14935, 139, 622, 11969, 30, 25, 12178, 111, 8331, 5866, 102, 52, 681, 3]


In [77]:
model([text, mask, segments])

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.7027308 ],
       [0.98811334],
       [0.9995449 ],
       [0.9718593 ]], dtype=float32)>

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 11683584    input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [ ]:
m = tf.keras.models.load_model("/content/drive/My Drive/AD/check/my_model")
print (m([text, mask, segments]) )
m.summary()


tf.Tensor(
[[0.00341263]
 [0.00178851]
 [0.996042  ]
 [0.89366364]], shape=(4, 1), dtype=float32)
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 11683584    input_word_ids[0][0]   

In [ ]:
resoult = m([text, mask, segments])

In [ ]:
resoult = resoult > 0.5
resoult = np.array(resoult, dtype='int')

In [ ]:
np.savetxt("foo.csv", resoult, delimiter=",")

In [75]:
model.save("/content/drive/My Drive/AD/check/my_model4")

INFO:tensorflow:Assets written to: /content/drive/My Drive/AD/check/my_model3/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/AD/check/my_model3/assets
